<a href="https://colab.research.google.com/github/nem-42098/SL_Proj_Unlearning/blob/main/Unlearn_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !git clone https://github.com/nem-42098/SL_Proj_Unlearning.git
# import os
# os.chdir('/content/SL_Proj_Unlearning')

In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from torchvision import models
import numpy as np


### Load Pre-Trained VGG network
> #### https://github.com/chenyaofo
> ### Note: There is some issue with using Batch Norm before ReLu as it creates a bias in the network. So people exchange the order between the two for tackling the bias

In [2]:
### First time when you wan to download the model
device=torch.device('cuda')
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg19_bn", pretrained=True)
model=model.to(device)
### For
#  future uses:Loading from the local

# model_1=torch.hub.load("C:/Users/nmura/.cache/torch/hub/chenyaofo_pytorch-cifar-models_master",'hubconf.py',source='local')



Using cache found in C:\Users\nmura/.cache\torch\hub\chenyaofo_pytorch-cifar-models_master


In [3]:
a=[[1,2,3],[2,3,4,5]]
import numpy as np
from itertools import chain
list(chain.from_iterable(a))

[1, 2, 3, 2, 3, 4, 5]

In [4]:
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): ReLU(inplace=True)
    (10): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): ReLU(inplace=True)
    (13): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (14): Conv2d(128, 256

In [3]:

def get_named_layers(net, is_state_dict=True):
    conv2d_idx = 0
    convT2d_idx = 0
    linear_idx = 0
    batchnorm2d_idx = 0
    named_layers = []
    for mod in net.modules():
        if isinstance(mod, torch.nn.Conv2d):
            layer_name = 'Conv2d{}_{}-{}'.format(
                conv2d_idx, mod.in_channels, mod.out_channels
            )
            named_layers.append(layer_name)
            if mod.bias is not None:
                named_layers.append(layer_name + '_bias')
            conv2d_idx += 1
        elif isinstance(mod, torch.nn.ConvTranspose2d):
            layer_name = 'ConvT2d{}_{}-{}'.format(
                conv2d_idx, mod.in_channels, mod.out_channels
            )
            named_layers.append(layer_name)
            if hasattr(mod, "bias"):
                named_layers.append(layer_name + '_bias')
            convT2d_idx += 1
        elif isinstance(mod, torch.nn.BatchNorm2d):
            layer_name = 'BatchNorm2D{}_{}'.format(
                batchnorm2d_idx, mod.num_features)
            named_layers.append(layer_name)
            named_layers.append(layer_name + '_bais')
            if is_state_dict:
                named_layers.append(layer_name + '_running_mean')
                named_layers.append(layer_name + '_running_var')
                named_layers.append(layer_name + '_num_bathes_tracked')
            batchnorm2d_idx += 1
        elif isinstance(mod, torch.nn.Linear):
            layer_name = 'Linear{}_{}-{}'.format(
                linear_idx, mod.in_features, mod.out_features
            )
            named_layers.append(layer_name)
            if hasattr(mod, "bias"):
                named_layers.append(layer_name + '_bias')
            linear_idx += 1
    return named_layers


In [7]:
a=np.eye(N=5)
a=a[:,0:3]
a=a.reshape(-1)
a

array([1., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

In [8]:
get_named_layers(model)

['Conv2d0_3-64',
 'Conv2d0_3-64_bias',
 'BatchNorm2D0_64',
 'BatchNorm2D0_64_bais',
 'BatchNorm2D0_64_running_mean',
 'BatchNorm2D0_64_running_var',
 'BatchNorm2D0_64_num_bathes_tracked',
 'Conv2d1_64-64',
 'Conv2d1_64-64_bias',
 'BatchNorm2D1_64',
 'BatchNorm2D1_64_bais',
 'BatchNorm2D1_64_running_mean',
 'BatchNorm2D1_64_running_var',
 'BatchNorm2D1_64_num_bathes_tracked',
 'Conv2d2_64-128',
 'Conv2d2_64-128_bias',
 'BatchNorm2D2_128',
 'BatchNorm2D2_128_bais',
 'BatchNorm2D2_128_running_mean',
 'BatchNorm2D2_128_running_var',
 'BatchNorm2D2_128_num_bathes_tracked',
 'Conv2d3_128-128',
 'Conv2d3_128-128_bias',
 'BatchNorm2D3_128',
 'BatchNorm2D3_128_bais',
 'BatchNorm2D3_128_running_mean',
 'BatchNorm2D3_128_running_var',
 'BatchNorm2D3_128_num_bathes_tracked',
 'Conv2d4_128-256',
 'Conv2d4_128-256_bias',
 'BatchNorm2D4_256',
 'BatchNorm2D4_256_bais',
 'BatchNorm2D4_256_running_mean',
 'BatchNorm2D4_256_running_var',
 'BatchNorm2D4_256_num_bathes_tracked',
 'Conv2d5_256-256',
 'Conv2

In [9]:
m = nn.Softmax(dim=1)
input = torch.randn(2, 3)
output = m(input)
output.data

tensor([[0.6047, 0.1680, 0.2273],
        [0.2125, 0.4605, 0.3270]])

### Check which pre-trained model are available:

In [4]:
torch.hub.list("chenyaofo/pytorch-cifar-models", force_reload=True)

Downloading: "https://github.com/chenyaofo/pytorch-cifar-models/zipball/master" to C:\Users\nmura/.cache\torch\hub\master.zip


['cifar100_mobilenetv2_x0_5',
 'cifar100_mobilenetv2_x0_75',
 'cifar100_mobilenetv2_x1_0',
 'cifar100_mobilenetv2_x1_4',
 'cifar100_repvgg_a0',
 'cifar100_repvgg_a1',
 'cifar100_repvgg_a2',
 'cifar100_resnet20',
 'cifar100_resnet32',
 'cifar100_resnet44',
 'cifar100_resnet56',
 'cifar100_shufflenetv2_x0_5',
 'cifar100_shufflenetv2_x1_0',
 'cifar100_shufflenetv2_x1_5',
 'cifar100_shufflenetv2_x2_0',
 'cifar100_vgg11_bn',
 'cifar100_vgg13_bn',
 'cifar100_vgg16_bn',
 'cifar100_vgg19_bn',
 'cifar100_vit_b16',
 'cifar100_vit_b32',
 'cifar100_vit_h14',
 'cifar100_vit_l16',
 'cifar100_vit_l32',
 'cifar10_mobilenetv2_x0_5',
 'cifar10_mobilenetv2_x0_75',
 'cifar10_mobilenetv2_x1_0',
 'cifar10_mobilenetv2_x1_4',
 'cifar10_repvgg_a0',
 'cifar10_repvgg_a1',
 'cifar10_repvgg_a2',
 'cifar10_resnet20',
 'cifar10_resnet32',
 'cifar10_resnet44',
 'cifar10_resnet56',
 'cifar10_shufflenetv2_x0_5',
 'cifar10_shufflenetv2_x1_0',
 'cifar10_shufflenetv2_x1_5',
 'cifar10_shufflenetv2_x2_0',
 'cifar10_vgg11_bn

### Downlaoding the Dataset and Creating the Dataloaders

In [5]:
### Transformation
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
       (0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)
    )
])
### Pytorch Datasets
train_dataset = torchvision.datasets.CIFAR10(
    root= './data', train = True,
    download =True, transform = transform)
test_dataset = torchvision.datasets.CIFAR10(
    root= './data', train = False,
    download =True, transform = transform)
### Dataloaders
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
len(train_dataset)

50000

### Create the Forget Dataset

In [6]:
# Define the classes
classes = ['forget', 'retain']

# Create a dictionary to store datasets for each class
class_datasets = {class_name: [] for class_name in classes}

# Iterate through the CIFAR-10 dataset and split it into class-specific subsets
for image, label in train_dataset:
  if label == 1:
    class_datasets['forget'].append((image, label))

  else:
      class_datasets['retain'].append((image, label))

# You now have class-specific subsets in the class_datasets dict


#### Forget and Retain

In [11]:
# Class split
retain_dataloader = torch.utils.data.DataLoader(class_datasets['retain'], batch_size=128, shuffle=True, num_workers=2)
forget_dataloader = torch.utils.data.DataLoader(class_datasets['forget'], batch_size=128, shuffle=True, num_workers=2)

# Random split
# train_split_dataset,forget_split_dataset=torch.utils.data.random_split(train_dataset,lengths=[45000,5000])
# retain_dataloader = torch.utils.data.DataLoader(train_split_dataset,  batch_size=128, shuffle=True, num_workers=2)
# forget_dataloader = torch.utils.data.DataLoader(forget_split_dataset, batch_size=128, shuffle=True, num_workers=2)

# Unlearner class


## Check that dumb model output random values
We expect this values to be uniform in [0,9], thus accuracy to be 10%.

In [7]:
# from tools.Unlearner import Unlearner

# unlearner = Unlearner(model, lr = 1e-6, alpha=1)
# ignorant = unlearner.reset_weights(model)

def test(model, dataloader):
  tp, n = 0,0
  for X,y in dataloader:
    X,y = X.to(device), y.to(device)

    with torch.no_grad():
      y_pred = model(X)

    tp += (y_pred.argmax(axis=1) == y).sum().item()
    n  += y.size(0)

  return tp/n

test(ignorant, forget_dataloader)

NameError: name 'ignorant' is not defined

In [9]:
### Congiguration file taken 
config={'max_epochs': 200,
                    
                     'data':
                      [('type_', 'cifar100'),
                                  ('image_size', 32),
                                  ('num_classes', 100),
                                  ('root', 'data/cifar100'),
                                  ('mean', [0.507, 0.4865, 0.4409]),
                                  ('std', [0.2673, 0.2564, 0.2761]),
                                  ('batch_size', 256),
                                  ('num_workers', 4)],
                     'model':[('type_', 'PyTorchHub'),
                                  ('force_reload', False),
                                  ('repo', 'chenyaofo/pytorch-cifar-models'),
                                  ('name', 'cifar100_vgg11_bn'),
                                  ('pretrained', False)],
                     'optimizer':
                      [('type_', 'SGD'),
                                  ('lr', 0.1),
                                  ('momentum', 0.9),
                                  ('dampening', 0),
                                  ('weight_decay', 0.0005),
                                  ('nesterov', True)],
                     'scheduler':
                      [('type_', 'CosineAnnealingLR'),
                                  ('T_max', 200),
                                  ('eta_min', 0)],
                     }

## Run the unlearning


In [15]:
from tools.Unlearner_DMM import UnlearnerDMM
model = torch.hub.load("chenyaofo/pytorch-cifar-models", "cifar10_vgg19_bn", pretrained=True)
model=model.to(device)
unlearner = UnlearnerDMM(model, config,alpha=0.2)


unlearner.unlearn(forget_dataloader,retain_dataloader)

Using cache found in C:\Users\nmura/.cache\torch\hub\chenyaofo_pytorch-cifar-models_master
  0%|          | 1/200 [00:39<2:10:02, 39.21s/it]

[0.12561849815221987, 1.0, 0.0]


  1%|          | 2/200 [01:21<2:15:29, 41.06s/it]

[0.0, 1.0, 0.0]


  2%|▏         | 3/200 [02:02<2:15:00, 41.12s/it]

...

[0.0, 1.0, 0.0]


  3%|▎         | 6/200 [04:19<2:25:28, 44.99s/it]

[0.0, 1.0, 0.0]


  3%|▎         | 6/200 [05:10<2:47:08, 51.70s/it]

[0.0, 1.0, 0.0]


In [16]:
test(unlearner.retained_model,retain_dataloader)

0.11237777777777777

In [17]:
test(unlearner.retained_model,forget_dataloader)

0.0

We can see that the unlearning process may need many epochs in the erasure phase to converge. We could also try to increase the learning rate for faster convergence.

In [ ]:
import pandas as pd
import seaborn as sns

log_df = pd.DataFrame.from_records(unlearner.log, columns=['phase', 'epoch', 'batch', 'tp', 'n', 'loss'])
unlearn_history = log_df.groupby(['phase', 'epoch']).agg({'tp':sum, 'n':sum, 'loss': 'mean'}).reset_index()
unlearn_history['accuracy'] = unlearn_history.tp / unlearn_history.n
unlearn_history.loc[unlearn_history.phase == 'erasure', 'loss_scale'] = unlearn_history.loc[unlearn_history.phase == 'erasure', 'loss'] /unlearn_history.loc[unlearn_history.phase == 'erasure', 'loss'].max() 
unlearn_history.loc[unlearn_history.phase == 'retrain', 'loss_scale'] = unlearn_history.loc[unlearn_history.phase == 'retrain', 'loss'] /unlearn_history.loc[unlearn_history.phase == 'retrain', 'loss'].max() 

sns.lineplot(data = unlearn_history, x = 'epoch', y='loss_scale', hue='phase')

## Check model performance
We now try check the model performance on the retain and forget set. 

In [ ]:
models = {'original':model, 'dumb':unlearner.dumb_model, 'erased':unlearner.erased_model, 'retrained':unlearner.retrained_model}
dataloaders = {'retain':retain_dataloader, 'forget':forget_dataloader, 'test':testloader}

performances = []
for model_name, m in models.items():
  for dl_name, dl in dataloaders.items():
    acc = test(m, dl)
    performances.append((model_name, dl_name, acc))

Strangely the dumb network has 0% performance on the forget set, and this may have a negative impact, note that the performance on the forget set is even too low (we expected 10%, not less).

Outside of that we note that this procedure works great for the retain and test which are fitted very nicely, and we can even see that there's a big leap in performance from the erased model to the retrained model.

In [ ]:
perf_df = pd.DataFrame.from_records(performances, columns=['model', 'data_partition', 'accuracy'])

tb = pd.pivot_table(perf_df, index='data_partition', columns='model', values='accuracy')
sns.heatmap(tb, annot = True, fmt='.2%')